## Imports

In [10]:
import polars as pl
import pandas as pd
import ta
from typing import List, Optional, Dict, Any, Union

# Custom modules
from data_loader import DataLoader
from config import BACKTEST_CONFIG

# Graph
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Get data

In [11]:
symbol, start_date, end_date, table_name = 'ftna', '2024-01-01', '2025-03-31', 'historical_summary'

# Initialize components
data_loader = DataLoader()

# Fetch data
market_data = data_loader.fetch_data(
    symbol=symbol,
    start_date=start_date,
    end_date=end_date,
    table=table_name
)

print(f"Loaded {len(market_data)} data points")

Loaded 311 data points


In [12]:
df = market_data.to_pandas()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    311 non-null    datetime64[us]
 1   open    311 non-null    object        
 2   high    311 non-null    object        
 3   low     311 non-null    object        
 4   close   311 non-null    object        
 5   volume  311 non-null    int64         
dtypes: datetime64[us](1), int64(1), object(4)
memory usage: 14.7+ KB


In [13]:
df['open'] = df['open'].astype(float)
df['high'] = df['high'].astype(float)
df['low'] = df['low'].astype(float)
df['close'] = df['close'].astype(float)

# df['open'].dtype()

In [14]:
# Sort the dataframe
df = df.sort_values('date')

In [15]:
df['sma_7'] = df['close'].rolling(window=7).mean()
df['sma_14'] = df['close'].rolling(window=14).mean()
df['sma_21'] = df['close'].rolling(window=21).mean()
df['sma_90'] = df['close'].rolling(window=90).mean()

In [16]:
df.head(20)

,date,open,high,low,close,volume,sma_7,sma_14,sma_21,sma_90
0,2024-01-02,10.01,10.18,10.01,10.08,26660,NaN,NaN,NaN,NaN
1,2024-01-03,10.08,10.18,10.00,10.08,120914,NaN,NaN,NaN,NaN
2,2024-01-04,10.08,10.60,10.00,10.21,83421,NaN,NaN,NaN,NaN
3,2024-01-05,10.21,10.60,10.01,10.50,127467,NaN,NaN,NaN,NaN
4,2024-01-08,10.50,10.67,10.01,10.33,87627,NaN,NaN,NaN,NaN
5,2024-01-09,10.33,10.60,10.00,10.15,164113,NaN,NaN,NaN,NaN
6,2024-01-10,10.15,10.61,10.12,10.59,66096,10.277143,NaN,NaN,NaN
7,2024-01-11,10.59,10.60,10.11,10.35,19343,10.315714,NaN,NaN,NaN
8,2024-01-12,10.35,10.75,10.50,10.63,73847,10.394286,NaN,NaN,NaN
9,2024-01-15,10.63,10.63,10.48,10.61,73121,10.451429,NaN,NaN,NaN


In [17]:
def create_candlestick_chart(
    df: pd.DataFrame, 
    date_col: str = 'date',
    open_col: str = 'open',
    high_col: str = 'high',
    low_col: str = 'low',
    close_col: str = 'close',
    sma_periods: List[int] = [10],
    additional_cols: Optional[List[str]] = None,
    title: str = 'Candlestick Chart with Indicators',
    height: int = 800,
    width: int = 1200,
    colors: Dict[str, str] = None
) -> go.Figure:
    """
    Create an interactive candlestick chart with SMA and additional indicators.
    
    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame containing the OHLC data
    date_col : str
        Column name for date values
    open_col, high_col, low_col, close_col : str
        Column names for OHLC data
    sma_periods : List[int]
        List of periods for SMA calculation
    additional_cols : List[str], optional
        Additional column names to plot as overlays
    title : str
        Chart title
    height, width : int
        Chart dimensions
    colors : Dict[str, str], optional
        Dictionary mapping column names to colors
        
    Returns:
    --------
    go.Figure
        Plotly figure object
    """
    # Ensure data is sorted by date
    df = df.sort_values(date_col)
    
    # Calculate SMAs if not already in DataFrame
    for period in sma_periods:
        col_name = f'sma_{period}'
        if col_name not in df.columns:
            df[col_name] = df[close_col].rolling(window=period).mean()
    
    # Create default colors if not provided
    if colors is None:
        colors = {}
    
    # Default SMA colors if not specified
    for period in sma_periods:
        col_name = f'sma_{period}'
        if col_name not in colors:
            # Create a gradient of blues for SMAs
            colors[col_name] = f'rgba(0, 0, 255, {0.5 + 0.5 * (sma_periods.index(period) / len(sma_periods))})'
    
    # Create the figure
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
    
    # Add candlestick trace
    fig.add_trace(
        go.Candlestick(
            x=df[date_col],
            open=df[open_col],
            high=df[high_col],
            low=df[low_col],
            close=df[close_col],
            name='Price'
        )
    )
    
    # Add SMA traces
    for period in sma_periods:
        col_name = f'sma_{period}'
        fig.add_trace(
            go.Scatter(
                x=df[date_col],
                y=df[col_name],
                line=dict(
                    color=colors.get(col_name, 'blue'),
                    width=1.5
                ),
                name=f'SMA {period}'
            )
        )
    
    # Add additional column traces
    if additional_cols:
        for col in additional_cols:
            if col in df.columns:
                fig.add_trace(
                    go.Scatter(
                        x=df[date_col],
                        y=df[col],
                        line=dict(
                            color=colors.get(col, 'purple'),
                            width=1.5
                        ),
                        name=col
                    )
                )
            else:
                print(f"Warning: Column '{col}' not found in DataFrame")
    
    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        height=height,
        width=width,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )
    
    return fig



In [18]:
fig = create_candlestick_chart(
    df, 
    sma_periods=[7, 14, 21, 90],
    #additional_cols=['rsi_14', 'macd'],
    colors={'sma_7': 'blue', 'sma_14': 'red', 'sma_21': 'green', 'sma_90': 'purple'}
)
fig.show()

In [18]:
df = ta.add_all_ta_features(
    df, open="open", high="high", low="low", close="close", volume="volume", fillna=True)

In [19]:
df.head()

,date,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2024-01-02,3.37,3.84,3.50,3.60,665397,-273987.000000,665397,-0.411765,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.600000,0.000000,0.000000,0.000000
1,2024-01-03,3.56,3.65,3.50,3.58,42735,-271138.000000,622662,-0.382892,-854.700000,...,-0.044336,-0.008867,-0.035469,-8.020852,-1.604170,-6.416682,3.561400,-0.555556,-0.557105,-0.555556
2,2024-01-04,3.70,3.69,3.55,3.64,573561,-107263.428571,1196223,-0.083689,4183.637143,...,0.054395,0.003785,0.050610,-7.416279,-2.766592,-4.649687,3.707090,1.675978,1.662088,1.111111
3,2024-01-05,3.54,3.64,3.50,3.59,133778,-69041.142857,1062445,-0.048776,2630.417551,...,0.020358,0.007100,0.013258,-13.290414,-4.871357,-8.419058,3.556823,-1.373626,-1.383148,-0.277778
4,2024-01-08,3.77,3.75,3.53,3.68,197936,2935.584416,1260381,0.001819,4799.535044,...,0.192524,0.044185,0.148339,-17.348023,-7.366690,-9.981333,3.661666,2.506964,2.476055,2.222222
